In [1]:
import pandas as pd
import numpy as np
import requests
import os
import psycopg2
import zipfile
import io
from sqlalchemy import create_engine

In [2]:
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD') 

In [3]:
url ='https://databank.worldbank.org/data/download/ESG_CSV.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [4]:
url = 'https://v-dem.net/media/datasets/V-Dem-CY-Core_csv_v13.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [5]:
vdem = pd.read_csv('V-Dem-CY-Core-v13.csv')
wb = pd.read_csv('ESGData.csv')
country = pd.read_csv('ESGCountry.csv')

### V-Dem

In [6]:
vdem_clean = vdem[['country_text_id', 'country_name', 'year', 'v2x_polyarchy']]

In [7]:
vdem_clean = vdem_clean.query("year >= 1960 and year <= 2021")

In [8]:
vdem_clean = vdem_clean.rename({'country_text_id': 'country_code',
                               'country_name': 'country_name_vdem',
                               'v2x_polyarchy':'democracy'}, axis = 1)

In [9]:
vdem_clean = vdem_clean.sort_values(by = ['country_code', 'year'])

In [10]:
vdem_clean.head(10)

,country_code,country_name_vdem,year,democracy
5433,AFG,Afghanistan,1960,0.080
5434,AFG,Afghanistan,1961,0.083
5435,AFG,Afghanistan,1962,0.082
5436,AFG,Afghanistan,1963,0.085
5437,AFG,Afghanistan,1964,0.137
5438,AFG,Afghanistan,1965,0.150
5439,AFG,Afghanistan,1966,0.161
5440,AFG,Afghanistan,1967,0.163
5441,AFG,Afghanistan,1968,0.163
5442,AFG,Afghanistan,1969,0.162


### ESG

In [11]:
country_clean = country[['Country Code', 'Table Name', 'Long Name', 'Currency Unit', 'Region', 'Income Group']]

In [12]:
country_clean = country_clean.rename({'Country Code': 'country_code',
                                     'Table Name': 'country_name_wb',
                                     'Long Name': 'country_longname', 
                                      'Currency Unit': 'currency_unit', 
                                      'Region': 'region', 
                                      'Income Group': 'income_group'}, axis = 1)

In [13]:
noncountries = ["Arab World", "Central Europe and the Baltics",
 "Caribbean small states",
 "East Asia & Pacific (excluding high income)",
 "Early-demographic dividend","East Asia & Pacific",
 "Europe & Central Asia (excluding high income)",
 "Europe & Central Asia", "Euro area",
 "European Union","Fragile and conflict affected situations",
 "High income",
 "Heavily indebted poor countries (HIPC)","IBRD only",
 "IDA & IBRD total",
 "IDA total","IDA blend","IDA only",
 "Latin America & Caribbean (excluding high income)",
 "Latin America & Caribbean",
 "Least developed countries: UN classification",
 "Low income","Lower middle income","Low & middle income",
 "Late-demographic dividend","Middle East & North Africa",
 "Middle income",
 "Middle East & North Africa (excluding high income)",
 "North America","OECD members",
 "Other small states","Pre-demographic dividend",
 "Pacific island small states",
 "Post-demographic dividend",
 "Sub-Saharan Africa (excluding high income)",
                "Sub-Saharan Africa",
 "Small states","East Asia & Pacific (IDA & IBRD)",
 "Europe & Central Asia (IDA & IBRD)",
 "Latin America & Caribbean (IDA & IBRD)",
 "Middle East & North Africa (IDA & IBRD)","South Asia",
 "South Asia (IDA & IBRD)",
 "Sub-Saharan Africa (IDA & IBRD)",
 "Upper middle income", "World"]

In [14]:
country_clean = country_clean.query('country_name_wb not in @noncountries')

In [15]:
country_clean.head(10)

,country_code,country_name_wb,country_longname,currency_unit,region,income_group
0,AFG,Afghanistan,Islamic State of Afghanistan,Afghan afghani,South Asia,Low income
1,AGO,Angola,People's Republic of Angola,Angolan kwanza,Sub-Saharan Africa,Lower middle income
2,ALB,Albania,Republic of Albania,Albanian lek,Europe & Central Asia,Upper middle income
3,AND,Andorra,Principality of Andorra,Euro,Europe & Central Asia,High income
5,ARE,United Arab Emirates,United Arab Emirates,U.A.E. dirham,Middle East & North Africa,High income
6,ARG,Argentina,Argentine Republic,Argentine peso,Latin America & Caribbean,Upper middle income
7,ARM,Armenia,Republic of Armenia,Armenian dram,Europe & Central Asia,Upper middle income
8,ATG,Antigua and Barbuda,Antigua and Barbuda,East Caribbean dollar,Latin America & Caribbean,High income
9,AUS,Australia,Commonwealth of Australia,Australian dollar,East Asia & Pacific,High income
10,AUT,Austria,Republic of Austria,Euro,Europe & Central Asia,High income


### World Bank

In [16]:
wb_clean = wb[['Country Code', 'Country Name', 'Indicator Code'] + [col for col in wb.columns if col.startswith('19') or col.startswith('20')]]

In [17]:
wb_clean = wb_clean.rename({'Country Code': 'country_code',
                           'Country Name': 'country_name_wb', 
                           'Indicator Code': 'feature'}, axis = 1)

In [18]:
noncountries.remove('World')

In [19]:
wb_clean = wb_clean.query('country_name_wb not in @noncountries')

In [20]:
replace_map = {
 "AG.LND.AGRI.ZS": "agricultural_land",
 "AG.LND.FRST.ZS": "forest_area",
 "AG.PRD.FOOD.XD": "food_production_index",
 "CC.EST": "control_of_corruption",
 "EG.CFT.ACCS.ZS": "access_to_clean_fuels_and_technologies_for_cooking",
 "EG.EGY.PRIM.PP.KD": "energy_intensity_level_of_primary_energy",
 "EG.ELC.ACCS.ZS": "access_to_electricity",
 "EG.ELC.COAL.ZS": "electricity_production_from_coal_sources",
 "EG.ELC.RNEW.ZS": "renewable_electricity_output",
 "EG.FEC.RNEW.ZS": "renewable_energy_consumption",
 "EG.IMP.CONS.ZS": "energy_imports",
 "EG.USE.COMM.FO.ZS": "fossil_fuel_energy_consumption",
 "EG.USE.PCAP.KG.OE": "energy_use",
 "EN.ATM.CO2E.PC": "co2_emissions",
 "EN.ATM.METH.PC": "methane_emissions",
 "EN.ATM.NOXE.PC": "nitrous_oxide_emissions",
 "EN.ATM.PM25.MC.M3": "pm2_5_air_pollution",
 "EN.CLC.CDDY.XD": "cooling_degree_days",
 "EN.CLC.GHGR.MT.CE": "ghg_net_emissions",
 "EN.CLC.HEAT.XD": "heat_index_35",
 "EN.CLC.MDAT.ZS": "droughts",
 "EN.CLC.PRCP.XD": "maximum_5-day_rainfall",
 "EN.CLC.SPEI.XD": "mean_drought_index","EN.MAM.THRD.NO": "mammal_species",
 "EN.POP.DNST": "population_density",
 "ER.H2O.FWTL.ZS": "annual_freshwater_withdrawals",
 "ER.PTD.TOTL.ZS": "terrestrial_and_marine_protected_areas",
 "GB.XPD.RSDV.GD.ZS": "research_and_development_expenditure",
 "GE.EST": "government_effectiveness",
 "IC.BUS.EASE.XQ": "ease_of_doing_business_rank",
 "IC.LGL.CRED.XQ": "strength_of_legal_rights_index",
 "IP.JRN.ARTC.SC": "scientific_and_technical_journal_articles",
 "IP.PAT.RESD": "patent_applications",
 "IT.NET.USER.ZS": "individuals_using_the_internet",
 "NV.AGR.TOTL.ZS": "agriculture",
 "NY.ADJ.DFOR.GN.ZS": "net_forest_depletion",
 "NY.ADJ.DRES.GN.ZS": "natural_resources_depletion",
 "NY.GDP.MKTP.KD.ZG": "gdp_growth",
 "PV.EST": "political_stability_and_absence_of_violence",
 "RL.EST": "rule_of_law",
 "RQ.EST": "regulatory_quality",
 "SE.ADT.LITR.ZS": "literacy_rate",
 "SE.ENR.PRSC.FM.ZS": "gross_school_enrollment",
 "SE.PRM.ENRR": "primary_school_enrollment",
 "SE.XPD.TOTL.GB.ZS": "government_expenditure_on_education",
 "SG.GEN.PARL.ZS": "proportion_of_seats_held_by_women_in_national_parliament",
 "SH.DTH.COMM.ZS": "cause_of_death",
 "SH.DYN.MORT": "mortality_rate",
 "SH.H2O.SMDW.ZS": "people_using_safely_managed_drinking_water_services",
 "SH.MED.BEDS.ZS": "hospital_beds",
 "SH.STA.OWAD.ZS": "prevalence_of_overweight",
 "SH.STA.SMSS.ZS": "people_using_safely_managed_sanitation_services",
 "SI.DST.FRST.20": "income_share_held_by_lowest_20pct",
 "SI.POV.GINI": "gini_index",
 "SI.POV.NAHC": "poverty_headcount_ratio_at_national_poverty_lines",
 "SI.SPR.PCAP.ZG": "annualized_average_growth_rate_in_per_capita_real_surve",
 "SL.TLF.0714.ZS": "children_in_employment",
 "SL.TLF.ACTI.ZS": "labor_force_participation_rate",
 "SL.TLF.CACT.FM.ZS": "ratio_of_female_to_male_labor_force_participation_ra",
 "SL.UEM.TOTL.ZS": "unemployment",
 "SM.POP.NETM": "net_migration",
 "SN.ITK.DEFC.ZS": "prevalence_of_undernourishment",
 "SP.DYN.LE00.IN": "life_expectancy_at_birth",
 "SP.DYN.TFRT.IN": "fertility_rate",
 "SP.POP.65UP.TO.ZS": "population_ages_65_and_above",
 "SP.UWT.TFRT": "unmet_need_for_contraception",
 "VA.EST": "voice_and_accountability",
 "EN.CLC.CSTP.ZS": "coastal_protection",
 "SD.ESR.PERF.XQ": "economic_and_social_rights_performance_score",
 "EN.CLC.HDDY.XD": "heating_degree_days",
 "EN.LND.LTMP.DC": "land_surface_temperature",
 "ER.H2O.FWST.ZS": "freshwater_withdrawal",
 "EN.H2O.BDYS.ZS": "water_quality",
 "AG.LND.FRLS.HA": "tree_cover_loss",
}

In [21]:
wb_clean['feature'] = wb_clean['feature'].map(replace_map)

In [22]:
wb_clean = pd.melt(wb_clean, id_vars = ['country_code', 'country_name_wb', 'feature'], value_vars = [str(i) for i in range(1960, 2023)])

In [23]:
wb_clean = wb_clean.rename({'variable' : 'year'}, axis = 1)

In [24]:
wb_clean = wb_clean.pivot(index=['country_code', 'country_name_wb', 'year'],
                                  columns='feature', values='value').reset_index()

In [25]:
wb_clean['year'] = wb_clean['year'].astype(int)

In [26]:
wb_clean

feature,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_surve,cause_of_death,...,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality
0,AFG,Afghanistan,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1961,NaN,NaN,57.878356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1962,NaN,NaN,57.955016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1963,NaN,NaN,58.031676,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1964,NaN,NaN,58.116002,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12217,ZWE,Zimbabwe,2018,30.0,45.400288,41.876696,7.319375,30.761677,NaN,NaN,...,NaN,-1.292463,406.23,5.0,27.214585,NaN,6.784,NaN,-1.136798,NaN
12218,ZWE,Zimbabwe,2019,30.2,46.682095,41.876696,9.819262,30.761677,NaN,47.647301,...,NaN,-1.303515,431.62,6.0,27.214747,NaN,7.370,NaN,-1.163669,NaN
12219,ZWE,Zimbabwe,2020,30.3,52.747667,41.876696,8.772859,30.761677,NaN,NaN,...,NaN,-1.329611,480.16,NaN,27.214747,NaN,7.898,NaN,-1.113408,83.3
12220,ZWE,Zimbabwe,2021,30.3,48.979927,41.876696,8.849899,NaN,NaN,NaN,...,NaN,-1.277202,NaN,NaN,27.214747,NaN,8.067,NaN,-1.135830,NaN


In [27]:
world_data = wb_clean[wb_clean['country_name_wb'] == 'World'].copy()

In [28]:
world_data.head(10).T

,11844,11845,11846,11847,11848,11849,11850,11851,11852,11853
feature,,,,,,,,,,
country_code,WLD,WLD,WLD,WLD,WLD,WLD,WLD,WLD,WLD,WLD
country_name_wb,World,World,World,World,World,World,World,World,World,World
year,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969
access_to_clean_fuels_and_technologies_for_cooking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
access_to_electricity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
tree_cover_loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unemployment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unmet_need_for_contraception,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
world_data.drop(['country_code', 'country_name_wb'], axis = 1)

feature,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_surve,cause_of_death,children_in_employment,co2_emissions,...,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality
11844,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11845,1961,NaN,NaN,35.879317,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11846,1962,NaN,NaN,35.952470,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11847,1963,NaN,NaN,36.035383,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11848,1964,NaN,NaN,36.117043,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11902,2018,67.696517,89.893733,36.738458,3.951747,9.137864,NaN,NaN,NaN,4.641294,...,2.196050,NaN,2.565871e+06,5.507853,14.337515,NaN,5.699076,NaN,NaN,NaN
11903,2019,68.921601,90.194716,36.762648,4.013242,9.167900,NaN,18.40885,NaN,4.582036,...,2.331934,NaN,2.734962e+06,5.685864,14.599801,NaN,5.543161,NaN,NaN,NaN
11904,2020,70.184805,90.483629,36.730920,4.352489,9.109511,NaN,NaN,NaN,4.291853,...,2.627081,NaN,2.933011e+06,NaN,14.787111,NaN,6.898273,NaN,NaN,NaN
11905,2021,71.331487,91.413941,36.841665,4.304093,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,14.552260,NaN,6.201871,NaN,NaN,NaN


In [30]:
world_data.columns

Index(['country_code', 'country_name_wb', 'year',
       'access_to_clean_fuels_and_technologies_for_cooking',
       'access_to_electricity', 'agricultural_land', 'agriculture',
       'annual_freshwater_withdrawals',
       'annualized_average_growth_rate_in_per_capita_real_surve',
       'cause_of_death', 'children_in_employment', 'co2_emissions',
       'coastal_protection', 'control_of_corruption', 'cooling_degree_days',
       'economic_and_social_rights_performance_score',
       'electricity_production_from_coal_sources', 'energy_imports',
       'energy_intensity_level_of_primary_energy', 'energy_use',
       'fertility_rate', 'food_production_index', 'forest_area',
       'fossil_fuel_energy_consumption', 'freshwater_withdrawal', 'gdp_growth',
       'ghg_net_emissions', 'gini_index', 'government_effectiveness',
       'government_expenditure_on_education', 'gross_school_enrollment',
       'heat_index_35', 'heating_degree_days', 'hospital_beds',
       'income_share_held_by_

In [31]:
new_column_names = {col: f"world_{col}" for col in world_data.columns if col != 'year'}
world_data_clean = world_data.rename(columns=new_column_names)
world_data_clean

feature,world_country_code,world_country_name_wb,year,world_access_to_clean_fuels_and_technologies_for_cooking,world_access_to_electricity,world_agricultural_land,world_agriculture,world_annual_freshwater_withdrawals,world_annualized_average_growth_rate_in_per_capita_real_surve,world_cause_of_death,...,world_research_and_development_expenditure,world_rule_of_law,world_scientific_and_technical_journal_articles,world_strength_of_legal_rights_index,world_terrestrial_and_marine_protected_areas,world_tree_cover_loss,world_unemployment,world_unmet_need_for_contraception,world_voice_and_accountability,world_water_quality
11844,WLD,World,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11845,WLD,World,1961,NaN,NaN,35.879317,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11846,WLD,World,1962,NaN,NaN,35.952470,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11847,WLD,World,1963,NaN,NaN,36.035383,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11848,WLD,World,1964,NaN,NaN,36.117043,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11902,WLD,World,2018,67.696517,89.893733,36.738458,3.951747,9.137864,NaN,NaN,...,2.196050,NaN,2.565871e+06,5.507853,14.337515,NaN,5.699076,NaN,NaN,NaN
11903,WLD,World,2019,68.921601,90.194716,36.762648,4.013242,9.167900,NaN,18.40885,...,2.331934,NaN,2.734962e+06,5.685864,14.599801,NaN,5.543161,NaN,NaN,NaN
11904,WLD,World,2020,70.184805,90.483629,36.730920,4.352489,9.109511,NaN,NaN,...,2.627081,NaN,2.933011e+06,NaN,14.787111,NaN,6.898273,NaN,NaN,NaN
11905,WLD,World,2021,71.331487,91.413941,36.841665,4.304093,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,14.552260,NaN,6.201871,NaN,NaN,NaN


In [32]:
wb_clean = wb_clean[wb_clean['country_name_wb'] != 'World']

In [33]:
wb_clean.head(10)

feature,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_surve,cause_of_death,...,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality
0,AFG,Afghanistan,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1961,NaN,NaN,57.878356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1962,NaN,NaN,57.955016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1963,NaN,NaN,58.031676,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1964,NaN,NaN,58.116002,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AFG,Afghanistan,1965,NaN,NaN,58.123668,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AFG,Afghanistan,1966,NaN,NaN,58.192662,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AFG,Afghanistan,1967,NaN,NaN,58.229459,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AFG,Afghanistan,1968,NaN,NaN,58.230992,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AFG,Afghanistan,1969,NaN,NaN,58.255523,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
